In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
!cp "/content/drive/My Drive/dataset/Customers.csv" /content/Customers.csv

In [81]:
!pip install catboost

#START HERE

In [82]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [83]:
df=pd.read_csv("/content/Customers.csv",)
df

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6
...,...,...,...,...,...,...,...,...
1995,1996,Female,71,184387,40,Artist,8,7
1996,1997,Female,91,73158,32,Doctor,7,7
1997,1998,Male,87,90961,14,Healthcare,9,2
1998,1999,Male,77,182109,4,Executive,7,2


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              2000 non-null   int64 
 1   Gender                  2000 non-null   object
 2   Age                     2000 non-null   int64 
 3   Annual Income ($)       2000 non-null   int64 
 4   Spending Score (1-100)  2000 non-null   int64 
 5   Profession              1965 non-null   object
 6   Work Experience         2000 non-null   int64 
 7   Family Size             2000 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 125.1+ KB


##Scaling Data

In [85]:
df = df.drop_duplicates()

In [86]:
from sklearn.preprocessing import RobustScaler

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

X_robust_scaled = df.copy()

scaler = RobustScaler()
X_robust_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df
X_robust_scaled

df['Gender'].value_counts()

,count
Gender,
Female,1186
Male,814


##One Hot Encoding Categorical Values

In [87]:
df=pd.get_dummies(df)
print(df.columns)

Index(['CustomerID', 'Age', 'Annual Income ($)', 'Spending Score (1-100)',
       'Work Experience', 'Family Size', 'Gender_Female', 'Gender_Male',
       'Profession_Artist', 'Profession_Doctor', 'Profession_Engineer',
       'Profession_Entertainment', 'Profession_Executive',
       'Profession_Healthcare', 'Profession_Homemaker', 'Profession_Lawyer',
       'Profession_Marketing'],
      dtype='object')


#Finding Outliers Using z Scores

In [88]:
from scipy.stats import zscore
import numpy as np


numerical_cols = df.select_dtypes(include=np.number).columns
z_scores = df[numerical_cols].apply(lambda x: np.abs(zscore(x, nan_policy='omit')))

threshold = 2.5
outlier_rows = df[(z_scores > threshold).any(axis=1)]

print("Rows identified as outliers based on Z-score:")
display(outlier_rows)

Rows identified as outliers based on Z-score:


,CustomerID,Age,Annual Income ($),Spending Score (1-100),Work Experience,Family Size,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing
42,43,48,22000,36,14,3,False,True,True,False,False,False,False,False,False,False,False
55,56,47,95000,41,14,2,False,True,True,False,False,False,False,False,False,False,False
150,151,43,30000,17,0,9,False,True,False,False,False,False,False,False,False,True,False
338,339,7,123804,28,14,1,True,False,False,True,False,False,False,False,False,False,False
353,354,23,181760,35,14,3,False,True,False,False,False,False,False,True,False,False,False
386,387,20,151124,18,15,3,True,False,True,False,False,False,False,False,False,False,False
387,388,25,67447,5,15,2,True,False,True,False,False,False,False,False,False,False,False
388,389,26,117454,8,16,2,True,False,False,False,False,False,False,True,False,False,False
392,393,21,119116,30,17,4,False,True,True,False,False,False,False,False,False,False,False
394,395,75,151696,82,15,1,True,False,True,False,False,False,False,False,False,False,False


#Handling NaN values

In [89]:
df = df.fillna(df.mean())

## splitting Dataset and Oversampling

In [90]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

X = df.drop(columns=['Gender_Female','Gender_Male'])
y = df[['Gender_Female','Gender_Male']]

X = df.iloc[:, :-3]
y_onehot = df.iloc[:, -3:]

y = y_onehot.idxmax(axis=1).astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

ros = RandomOverSampler(random_state=42) #smote gives better results

X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [91]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

## Making models

In [92]:
model = GradientBoostingClassifier(loss='log_loss', random_state=42)
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97       355
           1       0.41      0.61      0.49        28
           2       0.21      0.29      0.24        17

    accuracy                           0.89       400
   macro avg       0.54      0.61      0.57       400
weighted avg       0.93      0.89      0.91       400



In [93]:
model = GradientBoostingClassifier(loss='log_loss', random_state=42)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97       355
           1       0.42      0.64      0.51        28
           2       0.23      0.29      0.26        17

    accuracy                           0.90       400
   macro avg       0.55      0.63      0.58       400
weighted avg       0.93      0.90      0.91       400



In [94]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import re

# Sanitize column names
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'\[|\]|<', '_', col)
        new_cols.append(new_col)
    df.columns = new_cols
    return df

X_resampled = sanitize_col_names(X_resampled.copy())
X_test = sanitize_col_names(X_test.copy())

# Initialize and train the XGBoost model
# You might need to tune the hyperparameters for better performance
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_resampled, y_resampled)

X_test_aligned = X_test.reindex(columns=X_resampled.columns, fill_value=0)

y_pred_xgb = xgb_model.predict(X_test_aligned)

# Print classification report
print("Classification Report for XGBoost Model:")
print(classification_report(y_test, y_pred_xgb))

Classification Report for XGBoost Model:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       355
           1       0.44      0.64      0.52        28
           2       0.18      0.18      0.18        17

    accuracy                           0.90       400
   macro avg       0.54      0.59      0.56       400
weighted avg       0.92      0.90      0.91       400



In [95]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Initialize and train the CatBoost model
# CatBoost can handle categorical features directly, but our data is already one-hot encoded.
# You might need to tune hyperparameters for better performance
catboost_model = CatBoostClassifier(random_state=42, verbose=0) # verbose=0 to suppress training output
catboost_model.fit(X_resampled, y_resampled)

# Predict on the test set
y_pred_catboost = catboost_model.predict(X_test_aligned)

# Print classification report
print("Classification Report for CatBoost Model:")
print(classification_report(y_test, y_pred_catboost))

Classification Report for CatBoost Model:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       355
           1       0.46      0.64      0.54        28
           2       0.30      0.35      0.32        17

    accuracy                           0.91       400
   macro avg       0.59      0.65      0.61       400
weighted avg       0.93      0.91      0.92       400



In [96]:
catboost_model = CatBoostClassifier(random_state=42, verbose=0) # verbose=0 to suppress training output
catboost_model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_catboost = catboost_model.predict(X_test_aligned)

# Print classification report--evaluation technique
print("Classification Report for CatBoost Model:")
print(classification_report(y_test, y_pred_catboost))

Classification Report for CatBoost Model:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       355
           1       0.46      0.68      0.55        28
           2       0.28      0.29      0.29        17

    accuracy                           0.91       400
   macro avg       0.58      0.64      0.60       400
weighted avg       0.93      0.91      0.92       400

